In [0]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import TimeDistributed, LSTM, Dropout, Conv1D, Dense, Activation, MaxPooling1D, Input
from tensorflow.keras.utils import Sequence as Sequence
from tensorflow.keras.optimizers import SGD, Adam
import numpy as np
from google.colab import drive
from datetime import datetime
import ast
import time

try:
  import tensorflow.compat.v2 as tf
except Exception:
  pass

sample_rate = 100000
signals_count = 200
epochs = 10
batch_size = 128
capture_size = 2048

# Debug
#train_size = 16
#test_size = 4
#capture_size = 8
# Test
#train_size = 50
#test_size = 5
# Real
train_size = 1000
test_size = 200

dir_base_model = "/content/gdrive/My Drive/DeepLearning/sigid/test-"
print(tf.__version__)

2.1.0


In [0]:
drive.mount('/content/gdrive')

# Create a dtype with the binary data format and the desired column names
dt = np.dtype([('i', 'f4'), ('q', 'f4')])
# Load noisy data
data = np.fromfile(dir_base_model + "nothing.iq", dtype=dt)
dfa = pd.DataFrame(data)
dfa["label"] = "BLANK"
npArray = np.array(range(len(dfa)), dtype='f4', copy=True, order='K', subok=False, ndmin=0)
dfa["time"] = npArray / sample_rate
# Load FM data
data = np.fromfile(dir_base_model + "999.iq", dtype=dt)
dfb = pd.DataFrame(data)
dfb["label"] = "FM"
npArray = np.array(range(len(dfb)), dtype='f4', copy=True, order='K', subok=False, ndmin=0)
dfb["time"] = npArray / sample_rate

frames = [dfa, dfb]
result = pd.concat(frames)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
train = []
test = []

# Build datasets for train 
for i in (0, train_size):
  ii = i * capture_size
  train.append(dfa[ii:ii + capture_size].values)
for i in (0, train_size):
  ii = i * capture_size
  train.append(dfb[ii:ii + capture_size].values)

# Build datasets for test 
for i in (0, test_size):
  ii = i * capture_size
  test.append(dfa[ii:ii + capture_size].values)
for i in (0, test_size):
  ii = i * capture_size
  test.append(dfb[ii:ii + capture_size].values)

In [0]:
def get_signal_dataframes(df):
  interleave = 2
  dx = []
  dy = []
  tx = []
  ty = []

  # Generate train dataset 
  dfax = df[['i', 'q']] 
  dfay = df[['label']] 
  dfax = dfax * 10000.0
  for i in range(0, train_size):
    newx = dfax[interleave * i:interleave * i + capture_size]
    dx.append(newx)    
    newy = dfay[interleave * i:interleave * i + capture_size]
    dy.append(newy)

  # Offset to separate train from test datain df
  offset = train_size * interleave + capture_size

  print("Offset " + str(offset))

  # Generate test dataset 
  for i in range(0, test_size):
    newx = dfax[interleave * i + offset:interleave * i + capture_size + offset]
    tx.append(newx)
    newy = dfay[interleave * i + offset:interleave * i + capture_size + offset]
    ty.append(newy)

  # Convert lists to dataframes
  train_x = pd.DataFrame(dx) 
  train_y = pd.DataFrame(dy) 
  test_x = pd.DataFrame(tx) 
  test_y = pd.DataFrame(ty) 
  return train_x, train_y, test_x, test_y

def add_dataframe_for_signal(df, train_x, train_y, test_x, test_y):
  x, y, tx, ty = get_signal_dataframes(df)
  train_x.append(x)
  train_y.append(y)
  test_x.append(tx)
  test_y.append(ty)

train_x = []
train_y = []
test_x = []
test_y = []
add_dataframe_for_signal(dfa, train_x, train_y, test_x, test_y)
add_dataframe_for_signal(dfb, train_x, train_y, test_x, test_y)

# Build dataframes 
train_x = pd.concat(train_x)
train_y = pd.concat(train_y)
test_x = pd.concat(test_x)
test_y = pd.concat(test_y)

print("Initial X " + str(len(train_x.values)))
for i in range(len(train_x.values)):
  print(train_x.values[i][0].values[0])
print("")
print("Initial Y " + str(len(train_y.values)))
for i in range(len(train_y.values)):
  print(train_y.values[i][0].values[0])


# Shuffle data
permutations = np.random.permutation(len(train_y))
train_x = train_x.iloc[permutations]
train_y = train_y.iloc[permutations]
permutations = np.random.permutation(len(test_x))
test_x = test_x.iloc[permutations]
test_y = test_y.iloc[permutations]

print("Shuffled X " + str(len(train_x.values)))
for i in range(len(train_x.values)):
  print(train_x.values[i][0].values[0])
print("")
print("Shuffled Y " + str(len(train_y.values)))
for i in range(len(train_y.values)):
  print(train_y.values[i][0].values[0])

Se truncaron las últimas líneas 5000 del resultado de transmisión.
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['FM']
['

In [0]:
# Check saved accuracy
try:
    f = open(dir_base_model + 'best-noise-vs-fm-info-best.txt', 'rt')
    f.close()
    exists = True
except FileNotFoundError:
    exists = False
# Check if saved model is better. If false, save this one.
if (exists == True):
  file = open(dir_base_model + 'best-noise-vs-fm-info-best.txt', 'rt')
  value = file.read()
  file.close()
  old_evaluation_result = ast.literal_eval(value)
  print("Saved loss is: " + str(old_evaluation_result[0]))
  print("Saved accuracy is: " + str(old_evaluation_result[1]))

Saved loss is: 0.4870654909610748
Saved accuracy is: 0.771


In [0]:
def save_if_better(model):
  try:
      f = open(dir_base_model + 'best-noise-vs-fm-info-retrained.txt', 'rt')
      f.close()
      exists = True
  except FileNotFoundError:
      exists = False

  # Evaluate model
  evaluation_result = model.evaluate(test_x, test_y, verbose=1)
  # Check if saved model is better. If false, save this one.
  if (exists == True):
    file = open(dir_base_model + 'best-noise-vs-fm-info-retrained.txt', 'rt')
    value = file.read()
    file.close()
    old_evaluation_result = ast.literal_eval(value)
    print("Saved loss is: " + str(old_evaluation_result[0]))
    print("Saved accuracy is: " + str(old_evaluation_result[1]))

  # Save only if found a better configuration
  if (exists == False or old_evaluation_result[1] < evaluation_result[1]):
    print("Found better model with accuracy: " + str(evaluation_result[1]))
    #file = open(dir_base_model + 'best-noise-vs-fm-info.txt', 'wt')
    #file.write(str(evaluation_result))
    #file.close()
    #model.save(dir_base_model + 'best-noise-vs-fm-info-retrained.model')

In [0]:
def recreate_np_array_x(d, size):
  new = np.ndarray(shape=(signals_count * size, 2, capture_size), dtype=float)
  # train or test size i
  for i in range(len(d.values)):
    # 2048 j
    for j in range(len((d.values)[i])):
        new[i][0][j] = ((d.values)[i])[0].values[j][0]
        new[i][1][j] = ((d.values)[i])[0].values[j][1]
  return new

def recreate_np_array_y(d, size):
  new = np.ndarray(shape=(signals_count * size), dtype=np.int16)
  # 100 i
  for i in range(len(d.values)):
    result = ((d.values)[i])[0].values[0][0]
    if (result == 'FM'):
      new[i] = 1
    else:
      new[i] = 0
  return new

train_x = recreate_np_array_x(train_x, train_size)
train_y = recreate_np_array_y(train_y, train_size)
test_x = recreate_np_array_x(test_x, test_size)
test_y = recreate_np_array_y(test_y, test_size)

In [0]:
for i in range(len(train_y))
  print(str(i) + " " + str(train_y[i]))

[     0      1      0 ... -18240    254 -18432]


In [0]:
for i in range(len(test_y))
  print(str(i) + " " + str(test_y[i]))

[    1     0     1 ... 28515 30318 25649]


In [0]:
def load_last_and_train():
  model = tf.keras.models.load_model(dir_base_model + 'best-noise-vs-fm-info-best.model')
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
  start_time = time.time()
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  model.summary()
  model.fit(x = train_x, y = train_y, batch_size=batch_size, verbose=1, validation_data=(test_x, test_y), epochs=epochs)
  result = model.evaluate(test_x, test_y, verbose=0)
  end_time = time.time() - start_time
  print(str(result) + " taken " + str(end_time) + " seconds")
  return model, result

In [0]:
for x in range(1):
  model, result = load_last_and_train()
  save_if_better(model, result)

# Get final weights
#print("Current weights:\n")
#print(model.get_weights())

Model: "sequential_45"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_160 (Conv1D)          (None, 2, 10)             327690    
_________________________________________________________________
max_pooling1d_160 (MaxPoolin (None, 1, 10)             0         
_________________________________________________________________
conv1d_161 (Conv1D)          (None, 1, 10)             1210      
_________________________________________________________________
max_pooling1d_161 (MaxPoolin (None, 1, 10)             0         
_________________________________________________________________
conv1d_162 (Conv1D)          (None, 1, 10)             610       
_________________________________________________________________
max_pooling1d_162 (MaxPoolin (None, 1, 10)             0         
_________________________________________________________________
conv1d_163 (Conv1D)          (None, 1, 10)           

InvalidArgumentError: ignored

In [0]:
model, result = load_last_and_train()

In [0]:
print(str(result))

In [0]:
save_if_better(model, result)

In [0]:
result = model.predict()
if :
  print("This is a FM")
else
  print("This is a NOISE")

In [0]:
result = model.predict()
if (result):
  print("This is a FM")
else
  print("This is a NOISE")